In [1]:
# Access parent directories
from sys import path
from os.path import abspath
path += [abspath("../"*i) for i in (1, 2, 3)]

In [4]:
from QLab import Environment, SimpleAgent, QNetwork
import torch
from torch import nn
import torch.nn.functional as F

class DeepNetwork(QNetwork):
	"Simple and probably terrible triple layered qnetwork."
	def __init__(self, h, w):
		super().__init__()
		actions = 4
		self.conv1 = nn.Conv2d(4, 16, kernel_size=5, stride=2)
		self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
		self.connected1 = nn.Linear(32 * 4 * 5, 128)
		self.connected2 = nn.Linear(128, 4)


	# Called with either one element to determine next action, or a batch
	# during optimization. Returns tensor([[left0exp,right0exp]...]).
	def forward(self, x):
		x = F.relu(self.conv1(x))
		x = F.relu((self.conv2(x)))
		print(x.shape)
		x = torch.flatten(x, 1)
		x = F.relu(self.connected1(x))
		x = F.relu(self.connected2(x))
		return x

class RescaledRewardsAgent(SimpleAgent):
	def feedback(self, state, action, reward, new_state):
		if reward > 0: reward *= 100
		reward *= 1000
		reward -= 10
		self.memory.push(state, action, reward, new_state)
		self.optimizer(gamma = self.gamma)

height, width = Environment().shape
shnet = DeepNetwork(height, width)
agent = SimpleAgent(shnet)

In [5]:
agent.train(episodes=10, update_frq=3, plot=True, live=True)
agent.save()

  0%|          | 0/10 [00:00<?, ?it/s]

torch.Size([1, 32, 4, 5])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x640 and 9216x128)

In [ ]:
agent.train(live=True, episodes=10)

  0%|          | 0/10 [02:48<?, ?it/s]


error: display Surface quit